In [85]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
from cn_general_tool import *

In [86]:
sd = '2024-01-01 00:00:00.000'
ed = '2024-11-15 23:59:59.999'

instrument_list = ['000001.SZ', '000002.SZ', '600519.SH']

In [87]:
sql = f"""
WITH 
data_origin AS (
    SELECT
        date,
        instrument,
        make_date(year(date), month(date),  day(date)) AS day,
        AVG(close) OVER (PARTITION BY instrument, day ROWS BETWEEN  9 PRECEDING AND CURRENT ROW) AS ma_short,
        AVG(close) OVER (PARTITION BY instrument, day ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma_long,
        IF(ma_short > ma_long AND m_lag(ma_short,1) < m_lag(ma_long,1), 1, 0) AS signal_buy,
        IF(ma_short < ma_long AND m_lag(ma_short,1) > m_lag(ma_long,1), 1, 0) AS signal_sell,
    FROM cn_stock_bar1m_c
)
SELECT
    date,
    instrument,
    signal_buy,
    signal_sell
FROM data_origin
QUALIFY (signal_buy = 1 OR signal_sell = 1)
ORDER BY date, instrument
"""

In [88]:
df = dai.query(sql, filters={"date":[sd, ed], "instrument":instrument_list}).df()
df

,date,instrument,signal_buy,signal_sell
0,2024-01-02 09:56:00,600519.SH,0,1
1,2024-01-02 09:57:00,000001.SZ,0,1
2,2024-01-02 10:00:00,600519.SH,1,0
3,2024-01-02 10:01:00,600519.SH,0,1
4,2024-01-02 10:02:00,600519.SH,1,0
...,...,...,...,...
11689,2024-11-15 14:12:00,000001.SZ,0,1
11690,2024-11-15 14:30:00,000002.SZ,1,0
11691,2024-11-15 14:31:00,000001.SZ,0,1
11692,2024-11-15 14:34:00,000002.SZ,0,1


In [89]:
from bigmodule import M
import random

def cn_stock_strategy_signal_event_backtest(df, capital_base, holding_minutes):

    def BigTrader_Initialize(context):
        from bigtrader.finance.commission import PerOrder
        context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
        context.holding_minutes = holding_minutes
        context.weight = 1 / len(df['instrument'].unique())

    def BigTrader_Before_Trading(context, data):
        context.subscribe_bar(context.instruments, '1m')

    def BigTrader_Handle_Tick(context, tick):
        pass

    def BigTrader_Handle_Data(context, data):

        dt_now = data.current_dt.strftime("%Y-%m-%d %H:%M:%S")
        df_now = context.data[context.data["date"] == dt_now]

        instruments_hold = set(context.get_account_positions().keys())

        if dt_now[-8:] == "14:55:00":
            for instrument in instruments_hold:
                context.order_target_percent(instrument, 0) 

        if len(df_now) == 0:
            return

        instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
        instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
        

        for instrument in instruments_buy - instruments_hold:
            context.order_target_percent(instrument, context.weight)
        
        for instrument in instruments_sell:
            context.order_target_percent(instrument, 0)

        for instrument in instruments_hold:
            time_diff_minutes = (data.current_dt - context.get_position(instrument).last_sale_date).total_seconds() / 60
            if time_diff_minutes >= context.holding_minutes:
                context.order_target_percent(instrument, 0)

    def BigTrader_Handle_Trade(context, trade):
        pass

    def BigTrader_Handle_Order(context, order):
        pass

    def BigTrader_After_Trading(context, data):
        pass

    BigTrader = M.bigtrader.v34(
        
        data = df,
        
        start_date = """""",
        end_date   = """""",
        
        initialize           = BigTrader_Initialize,
        before_trading_start = BigTrader_Before_Trading,
        handle_tick          = BigTrader_Handle_Tick,
        handle_data          = BigTrader_Handle_Data,
        handle_trade         = BigTrader_Handle_Trade,
        handle_order         = BigTrader_Handle_Order,
        after_trading        = BigTrader_After_Trading,
        
        capital_base = capital_base  + random.uniform(0, 10),
        frequency="""minute""",
        product_type="""自动""",
        rebalance_period_type="""交易日""",
        rebalance_period_days="""1""",
        rebalance_period_roll_forward=True,
        backtest_engine_mode="""标准模式""",
        before_start_days=0,
        volume_limit=1,
        order_price_field_buy="""open""",
        order_price_field_sell="""close""",
        benchmark="""沪深300指数""",
        
        plot_charts=True,
        debug=False,
        backtest_only=False,
        m_name="""BigTrader"""
    ) 

    return BigTrader.raw_perf.read()

In [90]:
cn_stock_strategy_signal_event_backtest(df, capital_base=1000000, holding_minutes=5)

[2024-12-01 21:04:30] [info     ] bigtrader.v34 开始运行 ..
[2024-12-01 21:04:30] [info     ] 2024-01-02, 2024-11-15, , , instruments=3
[2024-12-01 21:04:30] [info     ] bigtrader module V2.2.0
[2024-12-01 21:04:30] [info     ] bigtrader engine v1.10.10 2024-11-26


INFO:MAIN:======== bigtrader pid:5047 version 1.10.10 2024-11-26 ========
INFO:MAIN:bigtrader run_mode:BACKTEST, handle_bar_mode:0, frequency:1m, exchange_mode:BQ2
INFO:MAIN:> process add account:BACKTEST,0,bkt000
INFO:ACCT[bkt000]:AccountEngine: self_calc:1, validate_self_trading:0, validate_cash:1, validate_position:1, enable_auto_planed_order:0
INFO:MAIN:login_account(bkt000)
INFO:MAIN:> add_strategy setting:{'strategy_name': 'strategy', 'account_id': 'bkt000'}
INFO:MAIN:init all strategy account_id:...
INFO:MAIN:stop all strategy account_id:...


[2024-12-01 21:05:32] [info     ] backtest done, raw_perf_ds:dai.DataSource("_3e3964f3aacc495a8a45ff7fb107148b")


[2024-12-01 21:05:33] [info     ] bigtrader.v34 运行完成 [62.795s].


,date,trading_days,portfolio_value,starting_value,ending_value,starting_cash,ending_cash,returns,algorithm_period_return,benchmark_period_return,...,today_sell_balance,commission,capital_changed,orders,transactions,positions,POS_FAC,TRA_FAC,LOG,benchmark_returns
2024-01-02,2024-01-02,0,987888.472712,0.0,820350.0,1.000003e+06,167538.472712,-0.012115,-0.012115,0.000000,...,0.0,249.66,0.0,"[{'symbol': '600519.SH', 'id': '1', 'dt': 2024...","[{'symbol': '600519.SH', 'dt': 2024-01-02 10:0...","[{'symbol': '600519.SH', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[10:00:00][id:...",0.000000
2024-01-03,2024-01-03,1,986182.852712,820350.0,0.0,1.675385e+05,986182.852712,-0.001727,-0.013820,-0.002379,...,819710.0,1065.62,0.0,"[{'symbol': '600519.SH', 'id': '4', 'dt': 2024...","[{'symbol': '600519.SH', 'dt': 2024-01-03 09:3...","[{'symbol': '600519.SH', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:25:00][id:...",-0.002379
2024-01-04,2024-01-04,2,986524.832712,0.0,820646.0,9.861829e+05,165878.832712,0.000347,-0.013478,-0.011605,...,0.0,246.02,0.0,"[{'symbol': '000001.SZ', 'id': '7', 'dt': 2024...","[{'symbol': '000001.SZ', 'dt': 2024-01-04 10:4...","[{'symbol': '000001.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[10:42:00][id:...",-0.009249
2024-01-05,2024-01-05,3,980716.752712,820646.0,0.0,1.658788e+05,980716.752712,-0.005887,-0.019286,-0.016903,...,1145629.0,1588.08,0.0,"[{'symbol': '000001.SZ', 'id': '10', 'dt': 202...","[{'symbol': '000001.SZ', 'dt': 2024-01-05 09:3...","[{'symbol': '000001.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:25:00][id:...",-0.005360
2024-01-08,2024-01-08,4,973314.622712,0.0,806611.0,9.807168e+05,166703.622712,-0.007548,-0.026688,-0.029618,...,0.0,244.13,0.0,"[{'symbol': '000001.SZ', 'id': '15', 'dt': 202...","[{'symbol': '000001.SZ', 'dt': 2024-01-08 09:3...","[{'symbol': '000001.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:25:00][id:...",-0.012933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11,2024-11-11,205,879113.652712,749758.0,0.0,1.367290e+05,879113.652712,-0.008318,-0.120889,0.219935,...,743351.0,966.36,0.0,"[{'symbol': '600519.SH', 'id': '718', 'dt': 20...","[{'symbol': '600519.SH', 'dt': 2024-11-11 09:3...","[{'symbol': '600519.SH', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:25:00][id:...",0.006599
2024-11-12,2024-11-12,206,864079.912712,0.0,727648.0,8.791137e+05,136431.912712,-0.017101,-0.135923,0.206532,...,0.0,222.74,0.0,"[{'symbol': '600519.SH', 'id': '721', 'dt': 20...","[{'symbol': '600519.SH', 'dt': 2024-11-12 09:4...","[{'symbol': '600519.SH', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:39:00][id:...",-0.010987
2024-11-13,2024-11-13,207,864216.572712,727648.0,0.0,1.364319e+05,864216.572712,0.000158,-0.135786,0.213958,...,728732.0,947.34,0.0,"[{'symbol': '600519.SH', 'id': '724', 'dt': 20...","[{'symbol': '600519.SH', 'dt': 2024-11-13 09:3...","[{'symbol': '600519.SH', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:35:00][id:...",0.006155
2024-11-14,2024-11-14,208,851103.632712,0.0,716924.0,8.642166e+05,134179.632712,-0.015173,-0.148899,0.192912,...,0.0,218.94,0.0,"[{'symbol': '000002.SZ', 'id': '727', 'dt': 20...","[{'symbol': '000002.SZ', 'dt': 2024-11-14 09:3...","[{'symbol': '000002.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:25:00][id:...",-0.017337
